<a href="https://colab.research.google.com/github/valeriazv/G3Patrones/blob/main/Challenges/Challege3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$$\Large \textit{C0708 | Reconocimiento de Patrones}$$

Autor: Emilio Ochoa


$$\large \textbf{Ejercicio practico | Training y Testing, Overfitting, regularization, Support Vector Machines}$$

1. Análisis exploratorio de datos

In [3]:
# Importamos librerías
import numpy as np
import pandas as pd
import math

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
%matplotlib inline

#Importamos el dataset de entrenamiento que es el que emplearemos
train_df = pd.read_csv('trainC3.csv', index_col='id')

# Para mostrar las primeras filas
train_df.head()

,sudden_fever,headache,mouth_bleed,nose_bleed,muscle_pain,joint_pain,vomiting,rash,diarrhea,hypotension,...,breathing_restriction,toe_inflammation,finger_inflammation,lips_irritation,itchiness,ulcers,toenail_loss,speech_problem,bullseye_rash,prognosis
id,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lyme_disease
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Tungiasis
2,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,Lyme_disease
3,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Zika
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,Rift_Valley_fever


In [4]:
# Para obtener información básica del DataFrame y un breve análisis estadístico:
print(train_df.info())
train_df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 707 entries, 0 to 706
Data columns (total 65 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sudden_fever           707 non-null    float64
 1   headache               707 non-null    float64
 2   mouth_bleed            707 non-null    float64
 3   nose_bleed             707 non-null    float64
 4   muscle_pain            707 non-null    float64
 5   joint_pain             707 non-null    float64
 6   vomiting               707 non-null    float64
 7   rash                   707 non-null    float64
 8   diarrhea               707 non-null    float64
 9   hypotension            707 non-null    float64
 10  pleural_effusion       707 non-null    float64
 11  ascites                707 non-null    float64
 12  gastro_bleeding        707 non-null    float64
 13  swelling               707 non-null    float64
 14  nausea                 707 non-null    float64
 15  chills

,sudden_fever,headache,mouth_bleed,nose_bleed,muscle_pain,joint_pain,vomiting,rash,diarrhea,hypotension,...,lymph_swells,breathing_restriction,toe_inflammation,finger_inflammation,lips_irritation,itchiness,ulcers,toenail_loss,speech_problem,bullseye_rash
count,707.000000,707.000000,707.000000,707.000000,707.000000,707.000000,707.000000,707.000000,707.000000,707.000000,...,707.000000,707.000000,707.000000,707.000000,707.000000,707.000000,707.000000,707.000000,707.000000,707.000000
mean,0.503536,0.449788,0.459689,0.487977,0.517680,0.449788,0.441301,0.487977,0.390382,0.393211,...,0.148515,0.072136,0.097595,0.079208,0.084866,0.154173,0.144272,0.137199,0.032532,0.031117
std,0.500341,0.497825,0.498725,0.500209,0.500041,0.497825,0.496894,0.500209,0.488181,0.488809,...,0.355861,0.258896,0.296977,0.270254,0.278879,0.361370,0.351614,0.344301,0.177533,0.173758
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


2. Preprocesamiento de la data

In [5]:
# Como podemos aprecias, la columna que usaremos para clasificar las enfermedades según su sintomatología; sin ebargo, esta está en formato 'string', por lo que empleamos el Encoder
from sklearn import preprocessing  # Para convertir variables categóricas

# Creamos una instancia
label_encoder = preprocessing.LabelEncoder()
train_df['prognosis'] = label_encoder.fit_transform(train_df['prognosis'])

train_df.head()

,sudden_fever,headache,mouth_bleed,nose_bleed,muscle_pain,joint_pain,vomiting,rash,diarrhea,hypotension,...,breathing_restriction,toe_inflammation,finger_inflammation,lips_irritation,itchiness,ulcers,toenail_loss,speech_problem,bullseye_rash,prognosis
id,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
2,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,3
3,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,6


In [6]:
#Creamos los grupos X, y
X = train_df.drop('prognosis', axis=1)
y = train_df['prognosis']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)

X_train.head()

,sudden_fever,headache,mouth_bleed,nose_bleed,muscle_pain,joint_pain,vomiting,rash,diarrhea,hypotension,...,lymph_swells,breathing_restriction,toe_inflammation,finger_inflammation,lips_irritation,itchiness,ulcers,toenail_loss,speech_problem,bullseye_rash
id,,,,,,,,,,,,,,,,,,,,,
66,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
367,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
178,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
y_train.head()

id
66      4
11     10
367     9
581     9
178     9
Name: prognosis, dtype: int64

3. Comparación de las métricas de evaluación

In [8]:
!gdown https://drive.google.com/uc?id=1KcIESuPAZilpnDAPKStt12g-KC3xpJcy

!pip -q install lazypredict

Downloading...
From: https://drive.google.com/uc?id=1KcIESuPAZilpnDAPKStt12g-KC3xpJcy
To: /content/cancer_challenge_cl2.csv
100% 123k/123k [00:00<00:00, 16.0MB/s]


In [9]:
from sklearn.model_selection import train_test_split # Para dividir los datos en conjuntos de entrenamiento y prueba.
from lazypredict.Supervised import LazyClassifier # Genera modelos de clasificación

clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)

# Evalua algoritmos de clasificación y proporciona una lista de modelos junto con las métricas de rendimiento estimadas.
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models



 97%|█████████▋| 28/29 [00:23<00:03,  3.80s/it]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000466 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 186
[LightGBM] [Info] Number of data points in the train set: 494, number of used features: 62
[LightGBM] [Info] Start training from score -2.418346
[LightGBM] [Info] Start training from score -2.331335
[LightGBM] [Info] Start training from score -2.142093
[LightGBM] [Info] Start training from score -2.538974
[LightGBM] [Info] Start training from score -2.706028
[LightGBM] [Info] Start training from score -2.835240
[LightGBM] [Info] Start training from score -2.270710
[LightGBM] [Info] Start training from score -2.290513
[LightGBM] [Info] Start training from score -2.091662
[LightGBM] [Info] Start training from score -2.513656
[LightGBM] [Info] Start training from score -2.488963
[LightGBM] [Warning] No further splits with positive 

100%|██████████| 29/29 [00:24<00:00,  1.17it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
NuSVC,0.33,0.34,None,0.31,0.22
BernoulliNB,0.31,0.33,None,0.27,0.03
NearestCentroid,0.31,0.33,None,0.27,0.04
SVC,0.31,0.32,None,0.28,0.29
RandomForestClassifier,0.30,0.31,None,0.29,0.51
RidgeClassifier,0.30,0.30,None,0.27,0.07
RidgeClassifierCV,0.29,0.29,None,0.27,0.11
LinearSVC,0.28,0.28,None,0.26,0.76
PassiveAggressiveClassifier,0.28,0.28,None,0.28,0.16


Según el análisis, el modelo a emplear para obtener la mejor precisión posible es nuSVC

4. Aplicación del método de clasificación:

In [19]:
from sklearn.svm import NuSVC

# Crea una instancia de SVC
nu_svc_classifier = NuSVC(nu=0.5, kernel='rbf', gamma='auto')


pipeline = Pipeline([
    ('scalar', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2)),
    ('nu_svc', nu_svc_classifier)
])

pipeline.fit(X_train, y_train)

print('Training score: {}'.format(pipeline.score(X_train, y_train)))
print('Test score: {}'.format(pipeline.score(X_test, y_test)))

Training score: 0.951417004048583
Test score: 0.3051643192488263


Como podemos ver, el modelo tiene problemas de Overfitting, debido a que es bastannte alta en precisión para la data de entrenamiento, mas no para la data de prueba. Es así que se decide hacer los arreglos L1 y L2 para simplificar el modelo.

###l2 penalization

In [54]:
steps = [
    ('scalar', StandardScaler()),
    ('model', Ridge(alpha=100, fit_intercept=True))
]

ridge_pipe = Pipeline(steps)
ridge_pipe.fit(X_train, y_train)

print('Training Score: {}'.format(ridge_pipe.score(X_train, y_train)))
print('Test Score: {}'.format(ridge_pipe.score(X_test, y_test)))

Training Score: 0.29106846151698995
Test Score: 0.14183863735498736


### l1 penalization

In [50]:
steps = [
    ('scalar', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2)),
    ('model', Lasso(alpha=0.3, fit_intercept=True))
]

lasso_pipe = Pipeline(steps)

lasso_pipe.fit(X_train, y_train)

print('Training score: {}'.format(lasso_pipe.score(X_train, y_train)))
print('Test score: {}'.format(lasso_pipe.score(X_test, y_test)))

Training score: 0.31743280540794283
Test score: 0.16926282708822293
